In [76]:
from recipe_scrapers import scrap_me
from bs4 import BeautifulSoup
import urllib
import threading
import time
import csv
import pandas
from multiprocessing.dummy import Pool as ThreadPool 

In [11]:
# url = 'https://www.allrecipes.com/recipes/728/world-cuisine/latin-american/mexican/?internalSource=hub%20nav&referringId=86&referringContentType=recipe%20hub&linkName=hub%20nav%20daughter&clickId=hub%20nav%202&page=2'
url = 'https://www.allrecipes.com/recipes/'

https://www.allrecipes.com/recipes/528 ignored!
https://www.allrecipes.com/recipes/514 ignored!
https://www.allrecipes.com/recipes/507 ignored!
https://www.allrecipes.com/recipes/542 ignored!
https://www.allrecipes.com/recipes/500 ignored!
https://www.allrecipes.com/recipes/508 ignored!
https://www.allrecipes.com/recipes/529 ignored!
https://www.allrecipes.com/recipes/543 ignored!
https://www.allrecipes.com/recipes/535 scrapped!
https://www.allrecipes.com/recipes/521 scrapped!
https://www.allrecipes.com/recipes/549 scrapped!
https://www.allrecipes.com/recipes/544 ignored!
https://www.allrecipes.com/recipes/536 ignored!
https://www.allrecipes.com/recipes/501 scrapped!
https://www.allrecipes.com/recipes/515 scrapped!
https://www.allrecipes.com/recipes/530 scrapped!
https://www.allrecipes.com/recipes/537 ignored!
https://www.allrecipes.com/recipes/509 scrapped!
https://www.allrecipes.com/recipes/522 scrapped!
https://www.allrecipes.com/recipes/531 ignored!
https://www.allrecipes.com/recip

https://www.allrecipes.com/recipes/667 ignored!
https://www.allrecipes.com/recipes/689 ignored!
https://www.allrecipes.com/recipes/646 ignored!
https://www.allrecipes.com/recipes/677 ignored!
https://www.allrecipes.com/recipes/653 ignored!
https://www.allrecipes.com/recipes/671 ignored!
https://www.allrecipes.com/recipes/683 ignored!
https://www.allrecipes.com/recipes/696 ignored!
https://www.allrecipes.com/recipes/690 ignored!
https://www.allrecipes.com/recipes/678 ignored!
https://www.allrecipes.com/recipes/672 ignored!https://www.allrecipes.com/recipes/684 ignored!

https://www.allrecipes.com/recipes/697 ignored!
https://www.allrecipes.com/recipes/691 ignored!
https://www.allrecipes.com/recipes/679 ignored!
https://www.allrecipes.com/recipes/685 ignored!
https://www.allrecipes.com/recipes/698 ignored!
https://www.allrecipes.com/recipes/673 ignored!
https://www.allrecipes.com/recipes/692 ignored!
https://www.allrecipes.com/recipes/680 ignored!
https://www.allrecipes.com/recipes/686 i

This cell works fine. It speeds up the scraping process by multithreading; however, if you run this it is highly likely that you'll be blocked by the target server.

In [ ]:
urls = []
for i in range(10000,700):
    urls.append( url+str(i) )
    
# Make the Pool of workers
pool = ThreadPool(4) 
# Open the urls in their own threads
# and return the results
results = pool.map(read_url, urls)
#close the pool and wait for the work to finish 
pool.close() 
pool.join() 

In [46]:
def read_url(url):
    try:
        html_doc = urllib.request.urlopen( url ).read()
#         pages.append(html_doc)
        print("{}".format(url)+" scrapped!")
        return html_doc
    except:
        print("{}".format(url)+" ignored!")
        pass

Instead we use a slower process; however, a more effective one.

In [13]:
#this is another approach taken to scrap allrecipes.com
url = 'https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page='
urls = []
pages = []
for i in range(1,2000):
    urls.append( url+str(i) )

for url in urls:
    try:
        html_doc = urllib.request.urlopen( url ).read()
        pages.append(html_doc)
        print("{}".format(url)+" scrapped!")
    except:
        print("last page is {} end".format(url))
        break

https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=1 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=2 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=3 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=4 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=5 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=6 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=7 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=8 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=9 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=10 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chinese/?page=11 scrapped!
https://www.allrecipes.com/recipes/695/world-cuisine/asian/chin

In [41]:
soups = []
for page in pages:
    soups.append( BeautifulSoup(page, 'html.parser') )

In [49]:
urls = []
for soup in soups:
    list_a_tags = soup.findAll("a", {"class": "fixed-recipe-card__title-link"})
    for item in list_a_tags:
        urls.append(item['href'])

In [51]:
res = []
for url in urls:
    res.append( read_url(url) )

https://www.allrecipes.com/recipe/215795/chinese-style-baby-bok-choy-with-mushroom-sauce/ scrapped!
https://www.allrecipes.com/recipe/214564/pan-fried-chinese-pancakes/ scrapped!
https://www.allrecipes.com/recipe/135367/chinese-lions-head-soup/ scrapped!
https://www.allrecipes.com/recipe/14759/pork-dumplings/ scrapped!
https://www.allrecipes.com/recipe/77408/chinese-spareribs/ scrapped!
https://www.allrecipes.com/recipe/8536/sweet-and-sour-chicken-i/ scrapped!
https://www.allrecipes.com/recipe/229242/fun-karnal-beef-and-broccoli/ scrapped!
https://www.allrecipes.com/recipe/49350/chinese-green-bean-stir-fry/ scrapped!
https://www.allrecipes.com/recipe/93234/honey-walnut-shrimp/ scrapped!
https://www.allrecipes.com/recipe/86230/szechwan-shrimp/ scrapped!
https://www.allrecipes.com/recipe/16954/chinese-chicken-fried-rice-ii/ scrapped!
https://www.allrecipes.com/recipe/9027/kung-pao-chicken/ scrapped!
https://www.allrecipes.com/recipe/15679/asian-beef-with-snow-peas/ scrapped!
https://www.

https://www.allrecipes.com/recipe/14758/spicy-pork-stir-fry/ scrapped!
https://www.allrecipes.com/recipe/152220/chicken-with-green-peppers-in-black-bean-sauce/ scrapped!
https://www.allrecipes.com/recipe/213146/honey-milk-tea-hong-kong-style/ scrapped!
https://www.allrecipes.com/recipe/128750/chinese-broccoli/ scrapped!
https://www.allrecipes.com/recipe/149229/taiwanese-style-three-cup-chicken/ scrapped!
https://www.allrecipes.com/recipe/97058/chinese-shrimp-and-tofu-soup/ scrapped!
https://www.allrecipes.com/recipe/92761/debs-general-tsos-chicken/ scrapped!
https://www.allrecipes.com/recipe/167942/hunan-kung-pao/ scrapped!
https://www.allrecipes.com/recipe/237807/crispy-ginger-beef/ scrapped!
https://www.allrecipes.com/recipe/223101/easy-shrimp-lo-mein/ scrapped!
https://www.allrecipes.com/recipe/12876/egg-rolls/ scrapped!
https://www.allrecipes.com/recipe/8914/chinese-chicken-salad/ scrapped!
https://www.allrecipes.com/recipe/174210/super-easy-stir-fried-cabbage/ scrapped!
https://ww

https://www.allrecipes.com/recipe/213472/he-jia-tuan-yuan-tofu-ball-soup-for-lunar-chinese-new-year/ scrapped!
https://www.allrecipes.com/recipe/218104/dong-po-chinese-pork-belly/ scrapped!
https://www.allrecipes.com/recipe/213335/shui-zhu-yu-sichuan-boiled-fish/ scrapped!
https://www.allrecipes.com/recipe/230861/grilled-kung-pao-shrimp/ scrapped!
https://www.allrecipes.com/recipe/53519/chinese-steamed-white-fish-fillet-with-tofu-cantonese-style/ scrapped!
https://www.allrecipes.com/recipe/56452/chicken-yum-yums/ scrapped!
https://www.allrecipes.com/recipe/55990/brandis-wontons/ scrapped!
https://www.allrecipes.com/recipe/74025/tofu-with-ground-pork-stir-fry/ scrapped!
https://www.allrecipes.com/recipe/70048/simple-and-tasty-chinese-steamed-buns/ scrapped!
https://www.allrecipes.com/recipe/67532/duck-sauce/ scrapped!
https://www.allrecipes.com/recipe/19514/steamed-garlic-prawns-chinese-style/ scrapped!
https://www.allrecipes.com/recipe/14792/chinese-new-year-turnip-cake/ scrapped!
http

https://www.allrecipes.com/recipe/246486/grandmas-hong-shau-rou-red-braised-pork/ scrapped!
https://www.allrecipes.com/recipe/246186/chinese-dongan-chicken/ scrapped!
https://www.allrecipes.com/recipe/246187/chicken-in-garlic-and-black-bean-sauce/ scrapped!
https://www.allrecipes.com/recipe/246188/sichuan-pork-stir-fry/ scrapped!
https://www.allrecipes.com/recipe/231420/dongan-chicken/ scrapped!
https://www.allrecipes.com/recipe/233736/chinese-tomato-and-egg-soup/ scrapped!
https://www.allrecipes.com/recipe/176531/wife-pastry-lo-po-bang/ scrapped!
https://www.allrecipes.com/recipe/233737/chinese-sesame-doughnuts/ scrapped!
https://www.allrecipes.com/recipe/233735/shrimp-wonton-with-noodles/ scrapped!
https://www.allrecipes.com/recipe/223589/la-pa-congee-porridge/ scrapped!
https://www.allrecipes.com/recipe/246481/slow-cooked-red-braised-pork-belly/ scrapped!
https://www.allrecipes.com/recipe/246184/pork-stuffed-bitter-melon/ scrapped!
https://www.allrecipes.com/recipe/246185/steamed-po

In [67]:
file = open("chinese.csv","w")
writer = csv.writer(file)
for row in res:
    soup = BeautifulSoup(row, 'html.parser')
    writer.writerow(soup)
file.close()

In [84]:
recepie = []
for data in res:
    soup = BeautifulSoup(data, 'html.parser')
    
    ingredients = soup.findAll("span", {"class": "recipe-ingred_txt added"})
    title = soup.findAll("h1",{"class":"recipe-summary__h1"})
    
    name = title[0].get_text()
    inge = []
    for ingredient in ingredients:
        inge.append( ingredient.get_text() )

    recepie.append({
        "name":name,
        "cuisine" : "chinese",
        "ingredient":inge
    })
    


In [85]:
ingredients = pandas.DataFrame(recepie)

In [86]:
ingredients

,cuisine,ingredient,name
0,chinese,"[2 tablespoons oyster sauce, 1 tablespoon soy ...",Chinese-Style Baby Bok Choy with Mushroom Sauce
1,chinese,"[1/4 teaspoon salt, 3/4 cup warm water, 1 cup ...",Pan-Fried Chinese Pancakes
2,chinese,"[1 pound ground pork, 1 egg, 1 tablespoon corn...",Chinese Lion's Head Soup
3,chinese,"[100 (3.5 inch square) wonton wrappers, 1 3/4 ...",Pork Dumplings
4,chinese,"[3 tablespoons hoisin sauce, 1 tablespoon ketc...",Chinese Spareribs
5,chinese,"[1 (8 ounce) can pineapple chunks, drained (ju...",Sweet and Sour Chicken I
6,chinese,"[2 1 1/2-inch-thick New York strip steaks, cut...",Fun Karnal (Beef and Broccoli)
7,chinese,"[1 cup vegetable oil, 13 pounds fresh green be...",Chinese Green Bean Stir-Fry
8,chinese,"[1 cup water, 2/3 cup white sugar, 1/2 cup wal...",Honey Walnut Shrimp
9,chinese,"[4 tablespoons water, 2 tablespoons ketchup, 1...",Szechwan Shrimp


In [87]:
ingredients.to_pickle("allrecipe_chinese")

In [88]:
ingredients.to_csv('allrecipe_chinese.csv')